# 3. Building Neural Networks with Keras
![LISA logo](https://raw.githubusercontent.com/wshand/Python-Data-Science-Workshop/master/assets/LISA_logo_medium.jpg)

This notebook helps introduce some of the most basic tools that are commonly used for doing data science and statistics in Python.

# Table of contents
* [Introduction](#introduction)
* [Additional resources](#additional-resources)

# Introduction <a id="introduction"></a>
One of the biggest developments in machine learning in recent years is the usage of [_**artificial neural networks**_](https://en.wikipedia.org/wiki/Artificial_neural_network) (ANNs) as a powerful tool in the data scientist's toolbox. Although neural nets have been around for over 70 years, they've only become popular in the past decade thanks to advancements in algorithms and hardware.

On the algorithms side, [backpropagation](https://en.wikipedia.org/wiki/Backpropagation) (aka backwards-mode automatic differentiation), introduced in the 1980s, made it much easier to train neural networks. Later, researchers introduced neural network architectures such as [convolutional networks](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNNs) and [LSTMs](https://en.wikipedia.org/wiki/Long_short-term_memory) that saw success in tasks such as handwriting recognition.

Meanwhile, hardware limitations originally made it difficult to train neural networks, which require more data and computing power than most machine learning methods. However, improvements in processor power and the advent of specialized hardware such as graphics processing units have made training neural nets much faster. Massively parallel computing has also enabled boosts in training time: for instance, Google's [AlphaZero](https://en.wikipedia.org/wiki/AlphaZero) was trained on 5,000 tensor processing units in parallel in just a few hours.

*Talk about recent achievements of NNs*

In this workshop, we will be using [Keras](https://keras.io) to build our own neural networks. Keras is a high-level interface for building ANNs, with libraries like [TensorFlow](https://www.tensorflow.org) running on the backend.

### Note: you will need to run this code cell every time you restart this notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy             as np

from keras.datasets import mnist
from keras          import backend
from keras.utils    import to_categorical
from keras.layers   import Dense, Dropout
from keras.layers   import Conv2D, MaxPooling2D, Flatten
from keras.models   import Sequential

########## Helper functions
def square_axes(ax, data, expansion_factor=1.05):
    # Change limits of plot axes to center on the input dataset, and to put the
    # x-axis and y-axis on the same scale
    m        = np.mean(data)
    max_dist = max([np.linalg.norm(u-m) for u in data]) * expansion_factor
    lims     = [m-max_dist, m+max_dist]
    try:    ax.set_xlim(lims); ax.set_ylim(lims)
    except: ax.xlim(lims); ax.ylim(lims)

def plot_decision_boundary(X, clf, ax, incr=1, h=.02):
    # Plot the support vector machine decision boundary for 2D data
    xmin, xmax = X[:,0].min()-incr, X[:,0].max()+incr
    ymin, ymax = X[:,1].min()-incr, X[:,1].max()+incr
    xx, yy     = np.meshgrid(np.arange(xmin,xmax,h),np.arange(ymin,ymax,h))
    Z          = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=.2)

def load_mnist_wrapper(n_classes=10):
    # Loads the MNIST dataset, and does some preprocessing
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # Only keep the first n_classes digits
    X_train = X_train[y_train < n_classes]
    y_train = y_train[y_train < n_classes]
    X_test  = X_test[y_test < n_classes]
    y_test  = y_test[y_test < n_classes]
    
    # Pre-process the data for the backend we're using
    if backend.image_data_format() == 'channels_first':
        X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
        X_test  = X_test.reshape(X_test.shape[0],   1, 28, 28)
    else:
        X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
        X_test  = X_test.reshape(X_test.shape[0],   28, 28, 1)

    # Change pixel intensities from the range 0 - 255 to the range 0 - 1
    X_train = X_train / 255
    X_test  = X_test  / 255

    # Convert the class labels into a format that Keras will be able to parse
    y_train = to_categorical(y_train, n_mnist_classes)
    y_test  = to_categorical(y_test, n_mnist_classes)
    
    return (X_train, y_train), (X_test, y_test)
##########################

# Constants
n_mnist_classes = 3

# Pre-load data
(X_train, y_train), (X_test, y_test) = load_mnist_wrapper(n_classes=n_mnist_classes)

# What are neural networks?
*Intro to neural nets*

Let's try building our first neural net. We're going to have a dataset that consists of two circles, one inscribed within the other. The points on the interior circle will be in the first class, and the points on the exterior circle will be in the second class.

In [ ]:
from sklearn.datasets import make_circles

X, classes = make_circles(n_samples=200, factor=0.3, noise=0.1)
plt.scatter(X[classes == 0,0], X[classes == 0,1], label="Class 0")
plt.scatter(X[classes == 1,0], X[classes == 1,1], label="Class 1")
plt.legend()
plt.show()

Our neural network will consist of an input layer, one hidden layer, and an output layer. Since our data are two-dimensional, we will have two neurons in the input layer (one for $x$ coordinates, another for $y$ coordinates). We'll put $8$ neurons in the hidden layer, and one neuron in the output layer. The output of the network will be an estimated probability that the data point is in the class on the interior circle.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

If you don't want to have to repeatedly call the `add` function, you can instead tell Keras what layers you want at the same time as you define your neural net:

In [ ]:
model = Sequential([
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

Now let's train the network! Our first step is to call `model.compile`. The `compile` function allows us to make some final configurations before we start training. We can do things like

* Set which [optimizer](https://keras.io/optimizers/) we'll use
* Choose a [loss function](https://keras.io/losses/), which helps us score how good our neural net is
* Set options that will be used on the backend (in this case, by TensorFlow) beneath the Keras API.

For more information on compilation options, see the [Keras](https://keras.io/getting-started/sequential-model-guide/#compilation) [documentation](https://keras.io/models/model/#compile).

Now that the model has been compiled, you can run `model.fit` to actually train the neural network. When you call `model.fit`, you provide a few parameters (such as `epochs` and `steps_per_epoch`) that tell Keras how much training you want to do. In the code below, we do $5$ training epochs each consisting of $512$ training steps, totalling $5\times 512 = 2560$ steps of neural network training.

In [ ]:
# Options:
#    loss='binary_crossentropy'
#      - Loss function for classification problems with two classes.
#      - If we had more than one class, we'd use 'categorical_crossentropy' instead
#
#    optimizer='rmsprop'
#      - Tells Keras to use the RMSProp optimization routine
#      - See https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
#
#    metrics=['accuracy']
#      - When we're training our neural net, Keras will tell us how accurate the net
#        currently is.
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# model.fit takes two numpy arrays. The first is the array X, which contains the actual
# data. The second is an array y that contains the predictions we are trying to make. Since
# this is a classification problem with two classes ("blue" and "orange"), y should contain
# 0's and 1's (where 0 = blue, 1 = orange).
#
# Options:
#     epochs=5
#       - Each 'epoch' can be thought of as a unit of training, consisting of
#         many small training steps
#       - The number of training steps in each epoch is controlled by 
#         steps_per_epoch
#
#     steps_per_epoch=512
#       - Number of training steps to do in each epoch
#       - The total number of training steps done when model.fit is called is 
#         epochs * steps_per_epoch
#     
#     verbose=1
#       - Tells Keras to show a progress bar for each training epoch that keeps us
#         updated on how much the network has been trained.
#
model.fit(X, classes, epochs=5, steps_per_epoch=512, verbose=1)

# Plot data, and the decision boundary found by the network
fig, axes = plt.subplots(1,2,figsize=(12,6))
for ax in axes:
    ax.scatter(X[classes == 0,0], X[classes == 0,1])
    ax.scatter(X[classes == 1,0], X[classes == 1,1])
    ax.set_xticks([]); ax.set_yticks([])

plot_decision_boundary(X, model, axes[1], incr=0.1)
plt.show()

Once your network has been trained, you can make predictions using `model.predict`, e.g.

```python
predictions = model.predict(Z)  # Z is some numpy array with data
```

# Building convolutional networks for the MNIST dataset
In this part of the workshop, we're going to start looking at how neural networks are used in computer vision. We're going to use the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database), a classic computer vision dataset containing images of tens of thousands of handwritten digits. The MNIST dataset can actually be loaded into Python using Keras:

```python
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
```

We will build a neural network that will be able to look at an MNIST digit tell us what number it sees. To help shorten training times we're only going to look at the digits $0$, $1$, and $2$; you can build a network that classifies more digits by increasing `n_mnist_classes` below.

In [ ]:
########################################################################
### For purpose of demonstration we're not going to use all of  ########
### the digits. Increase n_mnist classes if you want to try     ########
### training your neural net on more types of digits.           ########
########################################################################

n_mnist_classes = 3

########################################################################
########################################################################

# load_mnist_wrapper brings in the MNIST data with keras.datasets.mnist.load_data()
# and does some useful preprocessing. Check out the first code cell of this notebook
# to see how this works.
(X_train, y_train), (X_test, y_test) = load_mnist_wrapper(n_classes=n_mnist_classes)
    
# Show a few randomly selected images
fig, axes = plt.subplots(3, 5, figsize=(8,8))

c = np.random.choice(X_train.shape[0], axes.size)
digits, classes = X_train[c], y_train[c]

for (ax,img,num) in zip(axes.flatten(), digits, classes):
    ax.imshow(img.reshape((28,28)), cmap='gray')
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_xlabel("Digit: " + str(num.argmax()))

You could apply a plain feedforward neural network to this dataset, and you'd get decent results. 

In [ ]:
from sklearn.model_selection import KFold
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras.utils  import to_categorical
from keras        import backend

# Slight modification of architecture from the following example in the
# Keras repository:
# https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
mnist_model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation='relu'),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(.5),
    Dense(n_mnist_classes, activation='softmax')
])

mnist_model.compile(loss='categorical_crossentropy', optimizer='adadelta',
                    metrics=['accuracy'])
mnist_model.fit(X_train, y_train, verbose=1, batch_size=256,
                epochs=1, validation_data=(X_test, y_test))

score = mnist_model.evaluate(X_test, y_test, verbose=0)
print('Test loss: %.4f'     % score[0])
print('Test accuracy: %.4f' % score[1])

# If we didn't get 100% classification accuracy, show some images that we
# misclassified
predictions = mnist_model.predict(X_test).argmax(axis=1)
mclf_idx    = (predictions != y_test.argmax(axis=1)).flatten()
X_mclf      = X_test[mclf_idx]
y_mclf      = y_test[mclf_idx]
pred_mclf   = predictions[mclf_idx]

print("On test set, misclassified %d out of %d" % (X_mclf.shape[0], predictions.size))

if X_mclf.shape[0] >= 3:
    fig, axes   = plt.subplots(1,3,figsize=(8,8))
    c           = np.random.choice(X_mclf.shape[0], 3, replace=False)
    for (ii,ax) in enumerate(axes):
        ax.imshow(X_mclf[c[ii]].reshape((28,28)), cmap='gray')
        ax.set_xticks([]); ax.set_yticks([])
        pred_class, true_class = predictions[c[ii]], y_mclf[c[ii]].argmax()
        ax.set_xlabel("Predicted: " + str(pred_class) + 
                      "\nTrue class: " + str(true_class))

# Additional resources <a id="additional-resources"></a>
* [playground.tensorflow.org](https://playground.tensorflow.org/) allows you to experiment with some simple neural nets.
* [Hacker's guide to Neural Networks](https://karpathy.github.io/neuralnets/)
* References for specific ANN architectures:
  * Convolutional networks
  * LSTMs
    * [Understanding LSTMs](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)